In [ ]:
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow==2.0.0
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn
!pip3 install sentencepiece
!pip3 install tf_sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.0.0-rc1 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.0+zzzcolab20220506153740, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.4+zzzcolab20220516125453, 2.6.5, 2.6.5+zzzcolab20220523104206, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.0+zzzcolab20220506150900, 2.7.1, 2.7.2, 2.7.2+zzzcolab20220516114640, 2.7.3, 2.7.3+zzzcolab20220523111007, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.0+zzzcolab20220506162203, 2.8.1, 2.8.1+zzzcolab20220516111314, 2.8.1+zzzcolab20220518083849, 2.8.2, 2.8.2+zzzcolab20220523105045, 2.8.2+zzzcolab20220527125636, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1)
ERROR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import os
import re

import glob
#stop words
from nltk.corpus import stopwords
#tokenize
from nltk.tokenize import word_tokenize

from tqdm.auto import tqdm
from drive.MyDrive.pyrouge import Rouge

tqdm.pandas()

In [ ]:
#Stop word removal function
import gensim
all_stopwords = gensim.parsing.preprocessing.STOPWORDS

#from spacy.lang.en.stop_words import STOP_WORDS

def stop_word_remove(sentence):
    temp = [token for token in sentence.split() if token not in all_stopwords]
    return ' '.join(word for word in temp)

In [ ]:
path = '/content/drive/MyDrive/Parsed_Papers/Introduction/Introduction/S0167739X13001349.txt'

In [ ]:
def read_paper(path):
  f = open(path, 'r')
  text = str(f.read().strip())
  return text

In [ ]:
t = read_paper(path)

In [ ]:
def bodyMain(text):
  text = re.sub("@&#", " ", text)
  text = re.sub("\n", " ", text)
  text = (text.encode('ascii','ignore')).decode("utf-8")

  #Extract body from the paper

  body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text, flags= re.I)[0]

  # body = ' '.join(body_main.split())
  # body = body.split(".")
  return body_main

In [ ]:
x = bodyMain(t)
x

'   Highly-threaded, many-core devices such as GPUs have gained popularity in the last decade; both NVIDIA and AMD manufacture general purpose GPUs that fall in this category. The important distinction between these machines and traditional multi-core machines is that these devices provide a large number of low-overhead hardware threads with low-overhead context switching between them; this fast context-switch mechanism is used to hide the memory access latency of transferring data from slow large (and often global) memory to fast, small (and typically local) memory. Researchers have designed algorithms to solve many interesting problems for these devices, such as GPU sorting or hashing [14], linear algebra [57], dynamic programming [8,9], graph algorithms [1013], and many other classic algorithms [14,15]. These projects generally report impressive gains in performance. These devices appear to be here to stay. While there is a lot of folk wisdom on how to design good algorithms for the

In [ ]:
def extract(text):
  text = re.sub("@&#", " ", text)
  text = re.sub("\n", " ", text)
  text = (text.encode('ascii','ignore')).decode("utf-8")

  #Extract highlights, body from body of the paper 
  highlights = re.findall(r'HIGHLIGHTS(.*?)KEYPHASES', text, flags = re.I)[0]
  body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text, flags= re.I)[0]

In [ ]:
def abstract(text):
  text = re.sub("@&#", " ", text)
  text = re.sub("\n", " ", text)
  text = ' '.join(text.split())
  text = (text.encode('ascii','ignore')).decode("utf-8")

  abst = re.findall(r'ABSTRACT(.*?)INTRODUCTION', text, flags = re.I)[0]
  # print("abs_main: ",abst)
  return abst

In [ ]:
def process_paper(text):
  # Removes unwanted characters, accounting for unicode characters
    text = re.sub("@&#", " ", text)
    text = re.sub("\n", " ", text)
    text = ' '.join(text.split())
    text = (text.encode('ascii', 'ignore')).decode("utf-8")
    

  # Extracting the highlights, body from the paper
    highlights = re.findall(r'HIGHLIGHTS(.*?)KEYPHRASES', text,  flags = re.I)[0]
    abst = re.findall(r'.*(?:abstract)(.*?)introduction', text, flags=re.I)[0]
    body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text,  flags = re.I)[0]
    #body_main = re.findall(r'.*(?:abstract)(.*?)references', text, flags=re.I)[0]
    
    # Making a copy of the body, lowercasing body text, removing punctuations & extra spaces
    dummy_body = body_main.lower()
    dummy_body = re.sub('[^\w\s\d\.]','',dummy_body)
    dummy_body = ' '.join(dummy_body.split())
    dummy_body = dummy_body.split(".")

  # Removing extra spaces from the body text, which will be preserved to produce summaries
  # And splitting into sentences
    body = ' '.join(body_main.split())
    body = body.split(".")

  # Removing sentences that are too short or too long, as they wouldn't make apt summary text
    for i,x in enumerate(dummy_body):
        if (len(x.split())) < 3 or (len(x.split())) > 15: 
            
            dummy_body.pop(i)
            body.pop(i)
            
    # Making a copy of the abstract, lowercasing abstract text, removing punctuations & extra spaces
    dummy_abst = abst.lower()
    dummy_abst = re.sub('[^\w\s\d\.]','',dummy_abst)
    dummy_abst = ' '.join(dummy_abst.split())
    dummy_abst = dummy_abst.split(".")
    
  # Removing extra spaces from the abstract text, which will be preserved to produce summaries
  # And splitting into sentences
    ab = ' '.join(abst.split())
    ab = abst.split(".")
    
  # Removing sentences that are too short or too long, as they wouldn't make apt summary text
    for i,x in enumerate(dummy_abst):
        if (len(x.split())) < 3 or (len(x.split())) > 15: 
            
            dummy_abst.pop(i)
            ab.pop(i)

  # Making a copy of the highlights, lowercasing body text, removing punctuations & extra spaces
    dummy_highlights = highlights.lower()
    dummy_highlights = re.sub('[^\w\s\d]','',dummy_highlights)
    dummy_highlights = ' '.join(dummy_highlights.split())

  # Removing stop words from body & highlights
    body_copy = []
    for x in dummy_body:
      
        body_copy.append(stop_word_remove(x))
    highlight_copy = []
    for x in dummy_highlights.split():
        highlight_copy.append(stop_word_remove(x))
        
    # Removing stop words from abstarct
    abst_copy = []
    for x in dummy_abst:
      
        abst_copy.append(stop_word_remove(x))
        
    # Combing all of the abstract's sentences into one string    
    abst_copy = " ".join(sentence for sentence in abst_copy)
    abst_copy = " ".join(abst_copy.split())

  
  # Combing all of the highlights into one string    
    highlight_copy = " ".join(sentence for sentence in highlight_copy)
    highlight_copy = " ".join(highlight_copy.split())
    #print("BC",body_copy)
    
    return body_main, body_copy, highlights, highlight_copy, abst , abst_copy


In [ ]:
body_main, body_copy, highlights, highlight_copy, abst , abst_copy = process_paper(t)

In [ ]:
body_main

' Highly-threaded, many-core devices such as GPUs have gained popularity in the last decade; both NVIDIA and AMD manufacture general purpose GPUs that fall in this category. The important distinction between these machines and traditional multi-core machines is that these devices provide a large number of low-overhead hardware threads with low-overhead context switching between them; this fast context-switch mechanism is used to hide the memory access latency of transferring data from slow large (and often global) memory to fast, small (and typically local) memory. Researchers have designed algorithms to solve many interesting problems for these devices, such as GPU sorting or hashing [14], linear algebra [57], dynamic programming [8,9], graph algorithms [1013], and many other classic algorithms [14,15]. These projects generally report impressive gains in performance. These devices appear to be here to stay. While there is a lot of folk wisdom on how to design good algorithms for these

In [ ]:
body_copy

['important distinction machines traditional multicore machines devices provide large number lowoverhead hardware threads lowoverhead context switching fast contextswitch mechanism hide memory access latency transferring data slow large global memory fast small typically local memory',
 'projects generally report impressive gains performance',
 'devices appear stay',
 'interested analyzing characterizing performance algorithms highlythreaded manycore machines abstract algorithmic systematic manner',
 'years scientists designed models capture important aspects machines use',
 'model assumes operations including memory accesses unit time',
 'aggarwal vitter proposed disk access machine dam model 22 counts number memory transfers slow fast memory instead simply counting number memory accesses program',
 'number models consider memory access costs sequential algorithms different ways 2329',
 'pram model algorithms complexity analyzed terms workthe time taken algorithm 1 processor span call

In [ ]:
highlights

' We design a memory model to analyze algorithms for highly-threaded many-core systems. The model captures significant factors of performance: work, span, and memory accesses. We show the model is better than PRAM by applying both to 4 shortest paths algorithms. Empirical performance is effectively predicted by our model in many circumstances. It is the first formalized asymptotic model helpful for algorithm design on many-cores. '

In [ ]:
highlight_copy

'design memory model analyze algorithms highlythreaded manycore systems model captures significant factors performance work span memory accesses model better pram applying 4 shortest paths algorithms empirical performance effectively predicted model circumstances formalized asymptotic model helpful algorithm design manycores'

In [ ]:
abst

' A number of highly-threaded, many-core architectures hide memory-access latency by low-overhead context switching among a large number of threads. The speedup of a program on these machines depends on how well the latency is hidden. If the number of threads were infinite, theoretically, these machines could provide the performance predicted by the PRAM analysis of these programs. However, the number of threads per processor is not infinite, and is constrained by both hardware and algorithmic limits. In this paper, we introduce the Threaded Many-core Memory (TMM) model which is meant to capture the important characteristics of these highly-threaded, many-core machines. Since we model some important machine parameters of these machines, we expect analysis under this model to provide a more fine-grained and accurate performance prediction than the PRAM analysis. We analyze 4 algorithms for the classic all pairs shortest paths problem under this model. We find that even when two algorith

In [ ]:
abst_copy

'speedup program machines depends latency hidden number threads processor infinite constrained hardware algorithmic limits model important machine parameters machines expect analysis model provide finegrained accurate performance prediction pram analysis analyze 4 algorithms classic pairs shortest paths problem model example dense graphs dynamic programming algorithm johnsons algorithm performance pram model validate predictions model empirical measurements instantiation highlythreaded manycore machine nvidia gtx 480'

# **Load Model and Tokenizer**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language mo

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
import nltk
nltk.download('punkt')
def get_sentences(body_main):
  sentences = nltk.tokenize.sent_tokenize(body_main)
  return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sequence = get_sentences(body_main)
sequence

[' Highly-threaded, many-core devices such as GPUs have gained popularity in the last decade; both NVIDIA and AMD manufacture general purpose GPUs that fall in this category.',
 'The important distinction between these machines and traditional multi-core machines is that these devices provide a large number of low-overhead hardware threads with low-overhead context switching between them; this fast context-switch mechanism is used to hide the memory access latency of transferring data from slow large (and often global) memory to fast, small (and typically local) memory.',
 'Researchers have designed algorithms to solve many interesting problems for these devices, such as GPU sorting or hashing [14], linear algebra [57], dynamic programming [8,9], graph algorithms [1013], and many other classic algorithms [14,15].',
 'These projects generally report impressive gains in performance.',
 'These devices appear to be here to stay.',
 'While there is a lot of folk wisdom on how to design good

# **summary**

In [ ]:
test = glob.glob(f"/content/drive/MyDrive/Parsed_Papers/Introduction/Test_Files/*.txt")

In [ ]:
len(test)

40

In [ ]:
col_names = ['FileName','Abstract','Summary','Precision','Recall','F1_Score']
            #  'Actual_Score','Predicted_Score',]
# 'MAE','RMSE','R2_Score','Precision','Recall','F_Score']
result = pd.DataFrame(columns=col_names,dtype=object)

for i,path in enumerate(tqdm(test[:])):

  file_name = path.split('/')[-1].replace('.txt', '')
  text = read_paper(path)
  abstractL = abstract(text)
  body_main = bodyMain(text)

  sequence = get_sentences(body_main)

  def output(sequence):
    inputs = tokenizer.encode(sequence,
                            return_tensors='pt',
                            max_length=512,
                            is_split_into_words=True,
                            truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=80, length_penalty=5., num_beams=2)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary
  
  summary = output(sequence)
  
  r = Rouge()

  [precision, recall, f1_score] = r.rouge_l([abstractL], [summary])

  df = pd.DataFrame(columns=col_names,dtype=object)

  df.loc[i, ['FileName']] = file_name
  df.loc[i,['Abstract']] = abstractL
  df.loc[i,['Summary']] =  summary
  df.loc[i, ['Precision']] = precision
  df.loc[i, ['Recall']] = recall
  df.loc[i, ['F1_Score']] = f1_score

  result = pd.concat([result, df], axis = 0)

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
result

,FileName,Abstract,Summary,Precision,Recall,F1_Score
0,S0168874X14001413,The topic of the paper is related to the macr...,a finite element. a super-element. a crack-tip...,0.129167,0.808696,0.222755
1,S0168874X14001425,Multi-scale finite element (FE) modeling and ...,the other components in the frame are simulate...,0.239365,0.685455,0.354824
2,S0168874X14001437,Automotive door sealing system isolates passe...,the door seals are installed in narrow gaps be...,0.168074,0.807302,0.278225
3,S0168874X14001516,Topology optimization problem requires repeat...,topology optimization has been applied to dyna...,0.281382,0.664016,0.395267
4,S0168874X14001528,Accuracy and efficiency are significant facto...,finite element based techniques have been of g...,0.302857,0.622309,0.407432
5,S0168874X14001541,Bridging domain method (BDM) is a multiscale ...,atomistic-continuum interface should be minimi...,0.152991,0.704724,0.251405
6,S0168874X14001553,A modified interval parameter perturbation fi...,acoustic response is very sensitive to small c...,0.242653,0.675234,0.357011
7,S0168874X14001565,An Element-free Galerkin scaled boundary meth...,cyclically symmetrical structures are introduc...,0.372771,0.497835,0.426322
8,S0168874X14001577,"In this paper, the concept of configurational...",configurational force vectors are the gradient...,0.287712,0.689594,0.406024
9,S0168874X14001589,This paper deals with modeling and model redu...,a kinematic description of each layer as a pie...,0.217391,0.578358,0.316005


In [ ]:
result.to_csv('/content/drive/MyDrive/project/result/T5_Abstractive_Text_Summarization.csv')